In [2]:
from ihs_lei.data.load import DatabaseReflector
import pandas as pd

In [3]:
import sqlalchemy
class EconomicDataLoad(object):
    """docstring for EconomicDataLoad"""
    def __init__(self):
        self.db = DatabaseReflector()
        self.economic_table = self.db.reverse_table('t_economics')
        select = sqlalchemy.sql.select([self.economic_table])
        self.economic_data = pd.read_sql(select, self.db.engine)
        

In [4]:
economic_data = EconomicDataLoad()

In [5]:
dataset = economic_data.economic_data
dataset.head()

,quandl_code,date,value,indicator_name,country_name,country_code,indicator_code,start_date,frequency
0,ALBBCONF,2002-06-30,-7.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
1,ALBBCONF,2002-09-30,-2.7,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
2,ALBBCONF,2002-12-31,3.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
3,ALBBCONF,2003-03-31,1.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
4,ALBBCONF,2003-06-30,-4.1,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly


In [6]:
import ihs_lei
from pathlib import Path
module_init_path = Path(ihs_lei.__file__).parents[1]

In [7]:
countries = dataset[['country_name', 'country_code']]
country_to_country_code = countries.groupby('country_name').first().to_dict()['country_code']

In [8]:
indicator_per_country = dataset[dataset.frequency != 'Yearly']
print(indicator_per_country.head())
indicator_per_country_df = indicator_per_country[['country_name', 'quandl_code']].drop_duplicates().reset_index(drop=True)


  quandl_code        date  value       indicator_name country_name  \
0    ALBBCONF  2002-06-30   -7.2  Business Confidence      Albania   
1    ALBBCONF  2002-09-30   -2.7  Business Confidence      Albania   
2    ALBBCONF  2002-12-31    3.2  Business Confidence      Albania   
3    ALBBCONF  2003-03-31    1.2  Business Confidence      Albania   
4    ALBBCONF  2003-06-30   -4.1  Business Confidence      Albania   

  country_code indicator_code start_date  frequency  
0          ALB          BCONF 2002-06-30  Quarterly  
1          ALB          BCONF 2002-06-30  Quarterly  
2          ALB          BCONF 2002-06-30  Quarterly  
3          ALB          BCONF 2002-06-30  Quarterly  
4          ALB          BCONF 2002-06-30  Quarterly  


In [9]:
indicator_per_country = {}
for country in country_to_country_code.keys():
    indicator_per_country[country] = sorted(set(indicator_per_country_df[indicator_per_country_df['country_name'] == country]['quandl_code']))

In [10]:
freq_dict = {
    'Quarterly': 'Q',
    'Monthly': 'M',
    'Weekly': 'W',
    'Daily': 'B'
}

In [11]:
reference_series_list = ['IP', 'OIL']

In [12]:
from collections import defaultdict
from ihs_lei.analytics.filtering import *

indicator_fit_per_country = defaultdict(lambda: dict())
indicator_model_per_country = defaultdict(lambda: dict())
for country in sorted(country_to_country_code.keys()):
    print('[INFO] Working on {}'.format(country))
    country_exogenous_vars = indicator_per_country[country].copy()
    country_endog_vars = []
    for ref_series in reference_series_list:
        if ref_series in country_exogenous_vars:
            country_endog_vars.append(ref_series)
            country_exogenous_vars.remove(ref_series)
    for indicator_code in country_exogenous_vars:
        try:
            indicator_series = dataset[dataset['quandl_code'] == indicator_code]
            frequency = dataset[dataset['quandl_code'] == indicator_code]['frequency'].values[0]
            indicator_series.index = pd.to_datetime(indicator_series['date'])
            indicator_series = indicator_series['value'].asfreq(freq_dict[frequency], method='ffill').dropna()
            indicator_model_per_country[country][indicator_code] = SeasonalAdjustment(indicator_series)
            indicator_fit_per_country[country][indicator_code] = indicator_model_per_country[country][indicator_code].fit()
        except:
            print('[ERROR] Error in {} in indicator {}'.format(country, indicator_code))
            raise

[INFO] Working on Afghanistan
[ERROR] Error in Afghanistan in indicator AFGGOLD


TypeError: expected x and y to have same length

In [13]:
print(indicator_series.values.reshape((len(indicator_series.values),)))
print(dataset[dataset['quandl_code'] == 'AGOIP'])

[ 21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87
  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87
  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87  21.87]
      quandl_code        date  value         indicator_name country_name  \
18194       AGOIP  2011-03-31   -7.4  Industrial Production       Angola   
18195       AGOIP  2011-06-30   -9.0  Industrial Production       Angola   
18196       AGOIP  2011-09-30    1.1  Industrial Production       Angola   
18197       AGOIP  2011-12-31    6.3  Industrial Production       Angola   
18198       AGOIP  2012-03-31    6.0  Industrial Production       Angola   
18199       AGOIP  2012-06-30    7.0  Industrial Production       Angola   
18200       AGOIP  2012-09-30    8.8  Industrial Production       Angola   
18201       AGOIP  2012-12-31    1.7  Industrial Production       Angola   
18202       AGOIP  2013-03-31    3.1  Industrial Production       Angola   
18203       AGOIP  2013-06

In [18]:
print(sorted(set(dataset[dataset['country_name'] == 'Austria']['quandl_code'])))

['AUTBCONF', 'AUTBLR', 'AUTBOT', 'AUTCA', 'AUTCAGDP', 'AUTCARS', 'AUTCCONF', 'AUTCF', 'AUTCNCN', 'AUTCOP', 'AUTCPI', 'AUTCPIC', 'AUTCPICM', 'AUTCSP', 'AUTCTR', 'AUTCU', 'AUTCUR', 'AUTEDBT', 'AUTEMP', 'AUTEXVOL', 'AUTFER', 'AUTG', 'AUTGAGR', 'AUTGBGT', 'AUTGBVL', 'AUTGCP', 'AUTGD', 'AUTGDG', 'AUTGFCF', 'AUTGGR', 'AUTGNP', 'AUTGOLD', 'AUTGPC', 'AUTGPCP', 'AUTGSG', 'AUTGSP', 'AUTGYLD', 'AUTHCPI', 'AUTHOUS', 'AUTIBOR', 'AUTIMVOL', 'AUTIP', 'AUTIR', 'AUTJVAC', 'AUTLC', 'AUTLPS', 'AUTLUNR', 'AUTM1', 'AUTM2', 'AUTM3', 'AUTMANWG', 'AUTMKT', 'AUTMP', 'AUTMPMI', 'AUTNO', 'AUTPITR', 'AUTPOP', 'AUTPPI', 'AUTPPIC', 'AUTPSAV', 'AUTREM', 'AUTRSM', 'AUTRSY', 'AUTSSR', 'AUTSSRC', 'AUTSSRE', 'AUTSTR', 'AUTTOUR', 'AUTUNP', 'AUTUNR', 'AUTUNRY', 'AUTWAGE']
